<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Custom_Loss_Function_for_Fine_Control_in_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import models

class PerceptualLoss(torch.nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features
        self.model = torch.nn.Sequential(*list(vgg[:36]))  # Use the first 36 layers of VGG19
        self.layers = {'3': 'relu1_2', '8': 'relu2_2', '17': 'relu3_3', '26': 'relu4_3', '35': 'relu5_3'}  # Corrected layer names

    def forward(self, real, generated):
        real_feats = self.extract_features(real)
        gen_feats = self.extract_features(generated)
        loss = 0
        for layer in self.layers.values():
            loss += F.l1_loss(real_feats[layer], gen_feats[layer])
        return loss

    def extract_features(self, x):
        features = {}
        for name, layer in self.model._modules.items():
            x = layer(x)
            if name in self.layers:
                features[self.layers[name]] = x
        return features

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
perceptual_loss = PerceptualLoss().to(device)

real = torch.randn(1, 3, 224, 224).to(device)  # Example real image tensor
generated = torch.randn(1, 3, 224, 224).to(device)  # Example generated image tensor

loss = perceptual_loss(real, generated)
print(f"Perceptual Loss: {loss.item()}")